In [1]:
! pip install transformers
! pip install datasets
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import f1_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.1 MB/s 
     |████████████████████████████████| 120 kB 58.7 MB/s 
     |████████████████████████████████| 6.6 MB 52.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.3 MB/s 
     |████████████████████████████████| 115 kB 40.6 MB/s 
     |████████████████████████████████| 212 kB 46.2 MB/s 
     |████████████████████████████████| 127 kB 48.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:

from transformers import AutoTokenizer, AutoModel, AdamW
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
import time
# from torchmetrics import MeanSquaredError
import torch
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [3]:

import numpy as np

from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
from numpy import *
from torch import nn, optim

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation

Mounted at /content/drive
/content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation


In [5]:
df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
df2 = pd.read_json('/content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation/ERAI_Dataset_pairwise_test_with_translation.json')
df = df.dropna(axis = 0)

pd.set_option('display.max_rows', None)

df.reset_index(drop = True,inplace = True)

df['post1']=df['post1'].apply(lambda x:x.replace('\n',''))
df['post2']=df['post2'].apply(lambda x:x.replace('\n',''))

df['post1_en']=df['post1_en'].apply(lambda x:x.replace('\n',''))
df['post2_en']=df['post2_en'].apply(lambda x:x.replace('\n',''))


In [6]:
df_unsupervised = pd.read_json('/content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation/ERAI_Dataset_unsupervised_with_translation.json')

In [7]:
PRETRAINED_MODEL_NAME = "bert-base-chinese"
# 获取预测模型所使用的tokenizer
# AutoTokenizer, AutoModel,


#"bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [8]:

def generate_input_data(post,mpp_all_value,ml_all_value,max_len):

 
  
  all_value = pd.concat([mpp_all_value,ml_all_value],axis=1)
  
  
  full_text = []
  attention_mask = []
 
  for i in post:
    if len(i)>max_len:
      i = i[:max_len]
    
    i = tokenizer.tokenize(i)
    
    text = ['[CLS]']+i+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))
    
  all_pad_tokens = pad_sequences(full_text, maxlen=max_len, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len, padding='post', truncating='post')
 
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      
      'all_value': torch.tensor(all_value.values, dtype=torch.float),
      
  }

    

In [9]:
 class BertNerClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BertNerClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained("bert-base-chinese")
    self.drop = nn.Dropout(p=0.1)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask

    )[1]
    
    output = self.drop(output)
    # self.out(output)
    
    return self.out(output)

In [10]:

class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      # self.token_ids = data['token_type_ids']
      self.all_value = data['all_value']
     

    def __len__(self):

        return len(self.all_value)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.all_value[index]

In [11]:
def generate_dataloader(post,mpp_all_value,ml_all_value,max_len,batch_size):
  data = generate_input_data(post,mpp_all_value,ml_all_value,max_len)
  data = DataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  return dataloader


In [14]:
EPOCHS = 15


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_fn = nn.MSELoss().to(device)


label_num = 1
model= BertNerClassifier(label_num).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
def test_prediction(model, 
  test_dataloader, 
  loss_fn, 
  optimizer,
  device
  ):
  
  
  test_mpp_pred = []
  test_ml_pred = []
  real_mpp_label = []
  real_ml_label = []

  

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      
      targets = data[2].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        
      )
      

      
      for i in outputs:
        test_mpp_pred.append(i[0].item())
        test_ml_pred.append(i[1].item())
      for i in targets:
        real_mpp_label.append(i[0].cpu().item())
        real_ml_label.append(i[1].cpu().item())


      
  return test_mpp_pred,test_ml_pred,real_mpp_label,real_ml_label
     

In [22]:

def train_and_eval(model, 
  train_loader, 
  loss_fn, 
  optimizer,  
  device,
  epoch,
  label_num,
  test_dataloader,
  real_top_mpp,
  real_top_ml
  ):
  model.train()
  average_state = 1000
  
  for epo in range(epoch):
    
    
    train_loss = []
    
    train_predictions = []
    
    
   
    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      
      targets = data[2].to(device)
      

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        
      )
      

      loss = loss_fn(outputs, targets)
      loss = 10*loss
      
      

      train_loss.append(loss.item())
      
  
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    mpp_pred_value,ml_pred_value,real_mpp_label,real_ml_label= test_prediction(model, 
      test_dataloader, 
      loss_fn, 
      optimizer,
      device
      )

    

    
    pred_top_mpp = []
    pred_top_ml = []

    final_train_loss = np.mean(train_loss)



    real_mmp_average = np.mean(real_top_mpp[0:8])
    real_ml_average = np.mean(real_top_ml[0:8])

    
    pred_mmp_average = []
    pred_ml_average = []
    pred_mmp_rank = np.argsort(mpp_pred_value)[::-1][0:8]
    pred_ml_rank = np.argsort(ml_pred_value)[0:8]



    for i,j in zip(pred_mmp_rank,pred_ml_rank):
      pred_mmp_average.append(real_mpp_label[i])
      pred_ml_average.append(real_ml_label[j])

    pred_mmp_average = np.mean(pred_mmp_average)
    pred_ml_average = np.mean(pred_ml_average)

    mmp_difference = pred_mmp_average-real_mmp_average
    ml_difference = real_ml_average-pred_ml_average

    if (mmp_difference+ml_difference)/2 < average_state:
      accuracy_state = (mmp_difference+ml_difference)/2
      torch.save(model.state_dict(), 'task2_regression.bin')




    print(
'Epoch [{}] Train Totol Loss: {:.4f}, REAL AVERAGE MPP: {:.4F}, REAL AVERAGE ML: {:.4F}, PRED AVERAGE MPP: {:.4F} PRED AVERAGE ML: {:.4F}'
    .format(epo+1,final_train_loss,real_mmp_average,real_ml_average,pred_mmp_average,pred_ml_average))

In [ ]:

df = shuffle(df,random_state=42)

train = df[0:156]
  # validation = df[137:156]
test = df[156:200]

  # test = df[176:200]
train_post1 = train['post1']
train_post2 = train['post2']
train_mmp1 = train['MPP1']
train_mpp2 = train['MPP2']
train_ml1 = train['ML1']
train_ml2 = train['ML2']

all_train_post = pd.concat([train_post1,train_post2],axis=0)
all_train_mpp = pd.concat([train_mmp1,train_mpp2],axis=0)
all_train_ml = pd.concat([train_ml1,train_ml2],axis=0)

max_len = 512
batch_size = 8
train_dataloader = generate_dataloader(all_train_post,all_train_mpp,all_train_ml,max_len,batch_size)

test_post1 = test['post1']
test_post2 = test['post2']
test_mmp1 = test['MPP1']
test_mpp2 = test['MPP2']
test_ml1 = test['ML1']
test_ml2 = test['ML2']



all_test_post = pd.concat([test_post1,test_post2],axis=0)
all_test_mpp = pd.concat([test_mmp1,test_mpp2],axis=0)
all_test_ml = pd.concat([test_ml1,test_ml2],axis=0)

real_mpp_rank = np.argsort(all_test_mpp.to_list())
real_mpp_rank= real_mpp_rank[::-1]
real_ml_rank = np.argsort(all_test_ml.to_list())

real_top_mpp = []
real_top_ml = []

for i,j in zip(real_mpp_rank,real_ml_rank):
  
  real_top_mpp.append(all_test_mpp.tolist()[i])
  real_top_ml.append(all_test_ml.tolist()[j])


test_dataloader = generate_dataloader(all_test_post,all_test_mpp,all_test_ml,max_len,1)



label_num = 2
model= BertNerClassifier(label_num).to(device)

optimizer = AdamW(model.parameters(), lr=2e-6, correct_bias=False)
train_and_eval(model, 
    train_dataloader, 
    loss_fn, 
    optimizer,  
    device,
    EPOCHS,
    label_num,
    test_dataloader,
    real_top_mpp,
    real_top_ml
    
  )


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.load_state_dict(torch.load('task2_regression.bin', map_location=torch.device("cuda:0")))

<All keys matched successfully>

In [ ]:

def generate_test_data(post,max_len):
 
  
  full_text = []
  attention_mask = []
  
  for i in post:
    if len(i)>max_len:
      i = i[:max_len]
    
    i = tokenizer.tokenize(i)
    
    text = ['[CLS]']+i+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))
    
  all_pad_tokens = pad_sequences(full_text, maxlen=max_len, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len, padding='post', truncating='post')
  
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),

  }


In [ ]:

class testDataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index]

In [ ]:
def generate_test_dataloader(post,max_len,batch_size):
  data = generate_test_data(post,max_len)
  data = testDataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  return dataloader

In [ ]:
def test_prediction(model, 
  test_dataloader, 
  loss_fn, 
  optimizer,
  device
  ):
  
  
  test_pred = []
  

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      
      

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        
      )
      

      
      test_pred.append(outputs.cpu())
      


      
  return test_pred


In [ ]:
df_unsupervised

In [ ]:
post = df_unsupervised['post_rationale']
test_dataloader = generate_test_dataloader(post,512,1)

In [ ]:
test_pred = test_prediction(model, 
  test_dataloader, 
  loss_fn, 
  optimizer,
  device
  )

In [ ]:
mpp_pred_value = []
ml_pred_value = []
for i in test_pred:
  mpp_pred_value.append(i[0][0].item())
  ml_pred_value.append(i[0][1].item())
mpp_pred_value = np.argsort(mpp_pred_value)[::-1]
ml_pred_value = np.argsort(ml_pred_value)

In [ ]:
from pandas.core.frame import DataFrame

mpp=DataFrame(mpp_pred_value)
ml = DataFrame(ml_pred_value)


In [ ]:
mpp.to_json('ERAI_unsupervised_MPP_[aiml]_[1].json')
ml.to_json('ERAI_unsupervised_ML_[aiml]_[1].json')